In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import datetime
sns.set()

In [19]:
def select_fs_5y(stock_cd=[], account_nm=[], year=None, ci_div='IND'):
    if year is None:
        year = int(datetime.date.today().year)
    year_list = ','.join(str(x) for x in range(year-4, year+1))
    sql = "SELECT * FROM kor_fs WHERE year IN (" + year_list + ")"
    if not stock_cd:
        pass
    elif type(stock_cd) != list:
        raise TypeError('종목코드는 리스트 형태로 입력하여야합니다.')
    else:
        stock_cd_list = ','.join("'" + str(x) + "'" for x in stock_cd)
        sql = sql + " AND stock_cd IN (" + stock_cd_list + ")"
    if not account_nm:
        pass
    elif type(account_nm) != list:
        raise TypeError('계정과목명은 리스트 형태로 입력하여야합니다.')
    else:
        account_nm_list = ','.join("'" + str(x) + "'" for x in account_nm)
        sql = sql + " AND account_nm IN (" + account_nm_list + ")"
    sql = sql + " AND ci_div = '" + ci_div + "'"
    con = sqlite3.connect('./data/kor_stock.db')
    kor_fs = pd.read_sql(sql, con)
    con.close()
    return kor_fs

def roa5y_screener(year):
    kor_fs = select_fs_5y(year=year, account_nm=['당기순이익', '총자산'])
    kor_fs = kor_fs.pivot_table(index=['stock_cd', 'year'], columns='account_nm', values='fs_value')
    kor_fs['roa'] = kor_fs.당기순이익 / kor_fs.총자산
    kor_fs['roa'] = np.round(kor_fs['roa'], 4)
    kor_fs['roa'] = np.where(kor_fs.roa<=-1, np.nan, kor_fs.roa)    # roa가 (-)100% 이하인 경우 기하평균수익률 계산시 오류가 발생하므로 NA로 마스킹
    kor_fs = kor_fs.reset_index()
    kor_fs = kor_fs.pivot_table(index='stock_cd', columns='year', values='roa')
    kor_fs = kor_fs.dropna()    # 5년간 roa가 정상적으로 산출된 종목만 필터링
    kor_fs = kor_fs + 1
    kor_fs['roa5y'] = kor_fs.product(axis=1, skipna=True) ** (1 / kor_fs.count(axis=1))
    kor_fs['roa5y'] = np.round(kor_fs.roa5y, 4)
    kor_fs = kor_fs - 1
    kor_fs = kor_fs.sort_values('roa5y', ascending=False)
    kor_fs = kor_fs.reset_index()
    return kor_fs

def roic5y_screener(year):
    kor_fs = select_fs_5y(year=year, account_nm=['*총차입부채', '총자본', '현금및현금성자산', '*총금융자산', '투자부동산', '영업이익'])
    kor_fs = kor_fs.pivot_table(index=['stock_cd', 'year'], columns='account_nm', values='fs_value')
    kor_fs['ic'] = kor_fs['*총차입부채'] + kor_fs['총자본'] - kor_fs['현금및현금성자산'] - kor_fs['*총금융자산'] - kor_fs['투자부동산']
    kor_fs['ic'] = np.where(kor_fs.ic<=0, np.nan, kor_fs.ic)     #투하자본이 0이하인 종목은 아웃라이어로 보아 NA로 마스킹
    kor_fs['roic'] = kor_fs.영업이익 / kor_fs.ic
    kor_fs['roic'] = np.round(kor_fs.roic, 4)
    kor_fs['roic'] = np.where(kor_fs.roic<=-1, np.nan, kor_fs.roic)     # roic가 (-)100% 이하인 경우 기하평균수익률 계산시 오류가 발생하므로 NA로 마스킹
    kor_fs = kor_fs.reset_index()   # 5년간 roic가 정상적으로 산출된 종목만 필터링
    kor_fs = kor_fs.pivot_table(index='stock_cd', columns='year', values='roic')
    kor_fs = kor_fs.dropna()
    kor_fs = kor_fs + 1
    kor_fs['roic5y'] = kor_fs.product(axis=1, skipna=True) ** (1 / kor_fs.count(axis=1))
    kor_fs['roic5y'] = np.round(kor_fs.roic5y, 4)
    kor_fs = kor_fs - 1
    kor_fs = kor_fs.sort_values('roic5y', ascending=False)
    kor_fs = kor_fs.reset_index()
    return kor_fs

def listed_stock_filter_5y(year):
    con = sqlite3.connect('./data/kor_stock.db')
    sql = "SELECT * FROM kor_ticker WHERE fn_sec_nm != '금융'"  #금융회사 제외
    kor_ticker = pd.read_sql(sql, con)
    con.close()
    kor_ticker.listed_day = pd.to_datetime(kor_ticker.listed_day)
    kor_ticker.unlisted_day = pd.to_datetime(kor_ticker.unlisted_day)
    kor_ticker = kor_ticker[(kor_ticker.listed_day < pd.to_datetime(str(year-4)+'-01-01')) & 
                            ((kor_ticker.unlisted_day.isnull()) | 
                             (kor_ticker.unlisted_day > pd.to_datetime(str(year+1)+'-06-30')))]
    kor_ticker = kor_ticker[['stock_cd']]
    return kor_ticker

In [20]:
kor_ticker = listed_stock_filter_5y(2010)
kor_fs = roa5y_screener(2010)
roa5y_result = pd.merge(kor_ticker, kor_fs, how='left', on='stock_cd')
roa5y_result = roa5y_result.sort_values('roa5y', ascending=False)
roa5y_result.head(20)

,stock_cd,2006,2007,2008,2009,2010,roa5y
2,A035420,0.2766,0.3454,0.3622,0.3232,0.2790,0.3168
507,A072870,0.1845,0.2244,0.2436,0.2396,0.1763,0.2134
213,A002960,0.1499,0.2100,0.1638,0.3205,0.2282,0.2130
998,A032960,0.1746,0.1851,0.1479,0.2600,0.1771,0.1884
16,A033780,0.1771,0.1662,0.1841,0.1556,0.1772,0.1720
325,A018120,0.2041,0.1974,0.1807,0.1611,0.1189,0.1720
85,A058470,0.2113,0.1908,0.1594,0.1117,0.1721,0.1686
19,A010130,0.2235,0.1701,0.1304,0.1548,0.1539,0.1661
456,A065680,0.1398,0.1716,0.1213,0.2394,0.1577,0.1653
545,A054800,0.1707,0.2426,0.1916,0.1092,0.0996,0.1615


In [21]:
kor_ticker = listed_stock_filter_5y(2010)
kor_fs = roic5y_screener(2010)
roic5y_result = pd.merge(kor_ticker, kor_fs, how='left', on='stock_cd')
roic5y_result = roic5y_result.sort_values('roic5y', ascending=False)
roic5y_result.head(20)

,stock_cd,2006,2007,2008,2009,2010,roic5y
733,A057030,2.0221,1.1753,1.6576,1.3509,0.9513,1.4031
662,A036120,0.6742,0.7269,6.7784,0.3971,0.5404,1.1725
2,A035420,0.8463,0.8099,1.4416,0.8688,0.8786,0.9562
507,A072870,0.7819,0.8321,0.8461,1.1106,1.1245,0.9335
1145,A025850,0.2619,1.2255,3.5298,0.5531,0.3300,0.9227
611,A039340,0.9182,0.9315,0.8580,1.2436,0.5982,0.8988
363,A035510,0.5046,0.7108,0.8072,2.0485,0.5045,0.8443
387,A004960,3.1423,0.6851,0.1676,0.0488,0.9794,0.7607
37,A012750,0.5835,0.8429,0.9550,0.6584,0.7793,0.7589
421,A036190,0.2999,0.6614,1.0000,0.7571,1.2099,0.7576
